In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import dash 
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.figure_factory as ff

In [2]:
app = dash.Dash()

In [3]:
# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

confirmed_global=pd.read_csv(urls[0])
recovered_global=pd.read_csv(urls[2])
deaths_global=pd.read_csv(urls[1])
# extract dates
dates = confirmed_global.columns[4:]
dates
# melt dataframes into longer format
conf_df_long = confirmed_global.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_global.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recovered_global.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

recv_df_long = recv_df_long[recv_df_long['Country/Region']!='Canada']
# merge dataframes

full_table = pd.merge(left=conf_df_long, right=deaths_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
full_table = pd.merge(left=full_table, right=recv_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
temp = full_table.groupby(by=['Country/Region']).sum()
condition =temp['Recovered'] == 0
full_table['Recovered'] = full_table['Confirmed'] - full_table['Deaths']
full_table.drop(['Long', 'Lat'], axis=1, inplace=True)
full_table['Date'] = pd.to_datetime(full_table['Date'])
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, South', 'South Korea')
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, North', 'North Korea')
# Greenland
full_table.loc[full_table['Province/State']=='Greenland', 'Country/Region'] = 'Greenland'
#'Winter Olympics 2022' 'Summer Olympics 2020'
mask_ol_games = (full_table['Country/Region'] == 'Winter Olympics 2022') | (full_table['Country/Region'] == 'Summer Olympics 2020')
olympic_games = full_table[mask_ol_games]
full_table = full_table[~(mask_ol_games)]

# ship rows containing ships with COVID-19 reported cases
ships_rows = full_table['Province/State'].str.contains('Grand Princess') | \
            full_table['Province/State'].str.contains('Diamond Princess') | \
            full_table['Country/Region'].str.contains('Diamond Princess') | \
            full_table['Country/Region'].str.contains('MS Zaandam')

ships = full_table[ships_rows]

# Latest cases from the ships
ships_latest = ships[ships['Date']==max(ships['Date'])]
# ship_latest.style.background_gradient(cmap='Pastel1_r')

# skipping rows with ships info
full_table = full_table[~(ships_rows)]
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'
    # add 'WHO Region' column
full_table['WHO Region'] = full_table['Country/Region'].map(who_region)
full_table.fillna('Antarctica', inplace=True)
who_df = full_table.groupby('WHO Region')[['Confirmed', 'Deaths', 'Recovered']].agg('sum')
arab = ['Algeria', 'Bahrain', 'Comoros', 'Djibouti', 'Egypt', 'Iraq', 'Jordan', 'Kuwait', 'Lebanon', 'Libya', 'Mauritania', 'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi Arabia', 'Somalia', 'Sudan', 'Syria', 'Tunisia', 'United Arab Emirates', 'Yemen']
arab_df = full_table[full_table['Country/Region'].isin (arab)].reset_index()

In [4]:
top_con = full_table.groupby('Country/Region')['Confirmed'].sum().reset_index().sort_values('Confirmed', ascending=False).head(15)
top_con

,Country/Region,Confirmed
184,US,51743715124
80,India,28237359663
24,Brazil,20441906068
62,France,15309422519
66,Germany,12923214548
188,United Kingdom,11626106226
145,Russia,10139376823
86,Italy,9571614660
182,Turkey,8499888259
164,South Korea,7857495968


In [5]:
top_dth = full_table.groupby('Country/Region')['Deaths'].sum().reset_index().sort_values('Deaths', ascending=False).head(15)
top_dth

,Country/Region,Deaths
184,US,691491103
24,Brazil,474203381
80,India,354305846
114,Mexico,234425403
145,Russia,213221116
139,Peru,166361126
188,United Kingdom,156382009
86,Italy,124174772
62,France,110090847
38,Colombia,97825305


In [6]:
top_rec = full_table.groupby('Country/Region')['Recovered'].sum().reset_index().sort_values('Recovered', ascending=False).head(15)
top_rec

,Country/Region,Recovered
184,US,51052224021
80,India,27883053817
24,Brazil,19967702687
62,France,15199331672
66,Germany,12830516539
188,United Kingdom,11469724217
145,Russia,9926155707
86,Italy,9447439888
182,Turkey,8439109385
164,South Korea,7846954578


In [26]:
proportions = who_df.apply(lambda x: x/x.sum(), axis=0)
proportions

,Confirmed,Deaths,Recovered
WHO Region,,,
Africa,1.766065e-02,0.026743,1.753054e-02
Americas,3.235290e-01,0.450846,3.217052e-01
Antarctica,1.562454e-08,0.000000,1.584836e-08
Eastern Mediterranean,4.560132e-02,0.055185,4.546404e-02
Europe,3.898557e-01,0.316263,3.909099e-01
South-East Asia,1.206038e-01,0.120136,1.206105e-01
Western Pacific,1.027495e-01,0.030826,1.037799e-01


In [13]:
who_df.head()

,Confirmed,Deaths,Recovered
WHO Region,,,
Africa,5346390046,114338630,5232051416
Americas,97941613830,1927576623,96014037207
Antarctica,4730,0,4730
Eastern Mediterranean,13804841552,235939736,13568901816
Europe,118020605724,1352172252,116668433472


In [15]:
full_table.head()

,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered,WHO Region
0,Antarctica,Afghanistan,2020-01-22,0,0,0,Eastern Mediterranean
1,Antarctica,Albania,2020-01-22,0,0,0,Europe
2,Antarctica,Algeria,2020-01-22,0,0,0,Africa
3,Antarctica,Andorra,2020-01-22,0,0,0,Europe
4,Antarctica,Angola,2020-01-22,0,0,0,Africa


In [16]:
arab_df = arab_df.drop('WHO Region', axis=1)
arab_df.head()

,index,Province/State,Country/Region,Date,Confirmed,Deaths,Recovered
0,2,Antarctica,Algeria,2020-01-22,0,0,0
1,20,Antarctica,Bahrain,2020-01-22,0,0,0
2,94,Antarctica,Comoros,2020-01-22,0,0,0
3,107,Antarctica,Djibouti,2020-01-22,0,0,0
4,111,Antarctica,Egypt,2020-01-22,0,0,0


In [59]:
app.layout = html.Div(children=[
    html.H1('Covid-19 Dashboard'),
    html.Div([
        html.Div(children=[
        html.H3(children='Report Type'),
        dcc.Dropdown(
            id='report-type',
            options=[
                {'label': 'Global', 'value': 'global'},
                {'label': 'WHO Region', 'value': 'who'},
                {'label': 'Arab Contries', 'value': 'arab'}
            ],
            value='global'
        )
    ]),
    ]),
    html.Div([
    html.Div([], id='plot1'),

    html.Div([
        html.Div([], id='plot2'),
        html.Div([], id='plot3')
    ], style={'display': 'flex'}),
    html.Div([
        html.Div([], id='plot4')
    ])

])     
])

In [60]:
@app.callback([Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children'),
               Output(component_id='plot3', component_property='children'),
               Output(component_id='plot4', component_property='children')],
              Input(component_id='report-type', component_property='value'),
              [State("plot1", 'children'), State("plot2", "children"),
               State("plot3", "children"), State("plot4", "children")])
def update_graphs(report_type):
    if report_type == 'who':
        labels = proportions.index
        values_conf = proportions['Confirmed']
        values_dth = proportions['Deaths']
        values_rec = proportions['Recovered']        

        # Create the Plotly figure and display it
        confirmed_fig = go.Figure(data=[go.Pie(labels=labels, values=values_conf, textinfo='label+percent')],
                                  layout = go.Layout(title='Proportion of Confirmed COVID-19 Cases by WHO Region'))
        recovered_fig =go.Figure(data=[go.Pie(labels=labels, values=values_rec, textinfo='label+percent')],
                                 layout=go.Layout(title='Proportion of Recovered/Active COVID-19 Cases by WHO Region'))

        deaths_fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent')],
                               layout = go.Layout(title='Proportion of Recovered/Active COVID-19 Cases by WHO Region'))
        return confirmed_fig, recovered_fig, deaths_fig
    elif report_type == 'global':
        # Create the line chart for global data
        global_line_fig = px.line(full_table, x="Date", y=["Confirmed", "Deaths", "Recovered"], labels={"value": "No. of Cases"},
                                  title="The Change of Cases Over Time", xaxis_title="Date", yaxis_title="No. Of Cases")
        # Create the bar charts for top 10 countries
        
        top_confirmed_fig = go.Figure(data=[go.Bar(x=top_con['Country/Region'], y=top_con['Confirmed'])],
                layout = go.Layout(title='Top 15 Countries Affected',xaxis={'title':'Country'}, yaxis={'title':'Confirmed Cases'}))
        
        

        top_deaths_fig = go.Figure(data=[go.Bar(x=top_dth['Country/Region'], y=top_dth['Deaths'])],
               layout = go.Layout(title='Top 15 Countries in Deaths',xaxis={'title':'Country'}, yaxis={'title':'Deaths Cases'}))
        top_recovered_fig = go.Figure(data=[go.Bar(x=top_dth['Country/Region'], y=top_dth['Recovered'])],
               layout = go.Layout(title='Top 15 Countries in Recovered',xaxis={'title':'Country'}, yaxis={'title':'Recovered Cases'}))
        return global_line_fig, top_confirmed_fig, top_recovered_fig, top_deaths_fig
    else:
        # Create the line chart for Arab data
        arab_line_fig = px.line(arab_df, x="Date", y=["Confirmed", "Deaths", "Recovered/Active"], labels={"value": "No. of Cases"},
                                  title="The Change of Cases Over Time", xaxis_title="Date", yaxis_title="No. Of Cases")
        # Create the bar charts for each country
        country_confirmed_fig = go.Figure(data=[go.Bar(x=arab_df['Country/Region'],y=arab_df['Confirmed'])],
                                          layout = go.Layout(title='Covid-19 Confirmed Cases in Arab Countries',xaxis={'title':'Country'}, yaxis={'title':'Confirmed Cases'}))
        country_recovered_fig = go.Figure(data=[go.Bar(x=arab_df['Country/Region'], y=arab_df['Recovered'])],
                                          layout= go.Layout(title='Covid-19 Recovered in Arab Countries',xaxis={'title':'Country'}, yaxis={'title':'Recovered'}))
        country_deaths_fig = go.Figure(data=[go.Bar(x=arab_df['Country/Region'], y=arab_df['Deaths'])],
                                          layout = go.Layout(title='Covid-19 Deaths in Arab Countries',xaxis={'title':'Country'}, yaxis={'title':'Deaths'}))
        return arab_line_fig, country_confirmed_fig, country_recovered_fig, country_deaths_fig

In [63]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Feb/2023 04:11:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 04:11:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 04:11:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2023 04:11:15] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
